# Serial Communication for flashing SW - STM32 board flashing via UART

Protocol Sample (from Computer's standpoint).
Whenever the MCU sends a command to the computer, it'll wait for 1000ms for an answer. If not answered, the MCU will throw and error and/or try to proceed to the application phase, depending on the state of the comunication.

**Start of communication + Receive SW version and size:**
1. **MCU:     0x80 FF FF FF** -> PC Receives a 0x80 in the MSB, indicating that a MCU is present in the bootloader phase. Bytes 1,2,3 (LSB) don't care.
2. **PC:      0xC0 mj mn FF** -> Answers with a 0xC0 in the MSB (0x80 + 0x40), indicating that a serial communication is available and new SW might be flashed. The software MAJOR and MINOR version will be provided in bytes 1 and 2 respectively.
3. **MCU:     0x81 xx yy FF** -> MCU starts to erase each memory sector. The sector being currently erased is displayed through bit 1. Byte 3 will turn from zero (0) to one (1) when all app. dedicated sectors were erased.
4. **PC:      0xC1 xx xx xx** -> Answers with a 0xC1 in the MSB (0x81 + 0x40), indicating the number of packets that will be provided to flash the SW binary.
5. **MCU:     0x82 xx xx xx** -> MCU confirms that it can receive the provided packets (0x82). It'll also replicate the number of expected packets to receive in the LSB.

**Start of communication - Error Codes**
1. **MCU:     0x80 FF FF FF** -> Receives a 0x80 in the MSB, indicating that a MCU is present in the bootloader phase. Bytes 1,2,3 (LSB) don't care.
2. **PC:      0xC0 mj mn FF** -> Answers with a 0xC0 in the MSB (0x80 + 0x40), indicating that a serial communication is available and new SW might be flashed. The software MAJOR and MINOR version will be provided in bytes 1 and 2 respectively.
3. **MCU:     0x8E FF FF FF** -> MCU software matches the current software to be flashed. Skiping flash procedure.

---
**Binary data streaming (loop - normal operation):**
1. **PC:      0xyy yy yy yy** -> 4-byte package containing the SW binary.
2. **MCU:     0x83 xx 00 FF** -> MCU confirms that the given sector in 'byte 1' is starting to be flashed.
3. **MCU:     0x83 xx 01 FF** -> MCU confirms that the given sector in 'byte 1' is fully flashed.

**Binary data streaming (loop - error operation):**
1. **PC:      0xyy yy yy yy** -> 4-byte package containing the SW binary.
2. **MCU:     0x8F FF FF FF** -> Error - MCU couldn't flash the last packet provided / generic error.

---
**End of transmission**
1. **MCU:     0x84 00 xy zw** -> MCU informs transmission has been completed, and returns the checksum (CRC16) of the flashed binary in bytes 2 and 3.
expected number of bytes.

---
**Jump to application**
1. **MCU:     0x85 FF FF FF** ->MCU informs bootloader routine is finished, application SW is to be started.


### Configure Serial Communication

In [1]:
# ********** Configure Serial Comm **********
import serial
class SerialInterface():
    
    def __init__(self):
        SerialInterface.serialPort = serial.Serial(port="COM3", baudrate=576000, bytesize=8, timeout=0.001, stopbits=serial.STOPBITS_ONE)

    # Read 8-bytes package from serial buffer
    def readSerialBuffer(self, echo=1):
        if SerialInterface.serialPort.in_waiting > 0:
            serialString = SerialInterface.serialPort.read(4) # Read a 4-bytes stream
            SerialInterface.serialPort.reset_input_buffer() # Ignore all other data
            if echo == 1: print("     RX-> " , serialString.hex())
            return serialString
        else:
            return None
        
    # Write a hex string (ex.: "C0C1") to serial buffer
    def writeSerialBuffer(self,txBufferStr,echo=1):
        txBuffer = bytes.fromhex(txBufferStr)
        SerialInterface.serialPort.write(txBuffer)
        if echo == 1: print("     TX-> ", txBufferStr)

    # Close communication with serial port
    def closeSerialComm(self):
        print('     Stop serial pooling!')
        SerialInterface.serialPort.close()      

### Read binary file and create 4-byte packets

In [2]:
import math
import crcmod.predefined

class BinaryFile:
    BYTES_PER_PACKET = 4 # Read the binary and break it in pieces of "BYTES_PER_PACKET" size (32-bit)

    def __init__(self):
        self.fileData = []
        self.fileSizeBytes = 0
        self.fileSizePackets = 0
        self.filePackets = []

        self.SW_MAJOR = 0
        self.SW_MINOR = 0
        
        self.CRC_MSB = 0
        self.CRC_LSB = 0

    def loadBinary(self, path):
        self.fileData = open(path, mode="rb").read()
        self.fileSizeBytes = len(self.fileData) # File is parsed as "big-endian" - Not as expected by MCU
        self.fileSizePackets = math.ceil(len(self.fileData) / BinaryFile.BYTES_PER_PACKET)
        
        # Create array of data packets from the original binary
        # The packets are 32-bit elements (words) - we shall invert the byte order, because the original binary is stored as little endian, as per MCU architecture.
        for idx in range(self.fileSizePackets):
            try:
                self.filePackets.append(self.fileData[idx*BinaryFile.BYTES_PER_PACKET:(idx*BinaryFile.BYTES_PER_PACKET)+BinaryFile.BYTES_PER_PACKET][::-1]) # Split original binary in chunks of 32bits + invert bytes (get a 'big-endian' packet)
            except:
                print("filePacket - Error trying to packet position ", idx*BinaryFile.BYTES_PER_PACKET, " to ", (idx*BinaryFile.BYTES_PER_PACKET)+BinaryFile.BYTES_PER_PACKET, " Fallback to pack only the existing bytes.")
                self.filePackets.append(self.fileData[idx *BinaryFile.BYTES_PER_PACKET:][::-1])

        self.SW_MAJOR = self.filePackets[0x80].hex() # Pos. 0x200 of the binary (or packet 0x80) carries the SW Major Version
        self.SW_MINOR = self.filePackets[0x81].hex() # Pos. 0x204 of the binary (or packet 0x81) carries the SW Major Version

        # CRC Calculation - convert raw filedata to little endian
        crc_ccitt_16 = crcmod.predefined.Crc('crc-ccitt-false')
        crc_ccitt_16.update(self.fileData)
        self.CRC_MSB = ((crc_ccitt_16.crcValue & 0xFF00)>>8)
        self.CRC_LSB = ((crc_ccitt_16.crcValue & 0x00FF)) 

        # Print binary data for user
        print("--> Loading Binary file to be flashed <--")
        print("--> File Size: ", self.fileSizeBytes, " <--")
        print("--> Nr. of packets to be transmitted: ", self.fileSizePackets, " <--")
        print("--> Packets example: ", self.filePackets[0].hex() , " - ",self.filePackets[1].hex() , " - ",self.filePackets[2].hex() )
        print("--> SW Version: ", self.SW_MAJOR , " - ", self.SW_MINOR) # Display to the user
        print(f"--> CRC-CCITT (16-bit) is: 0x{format(crc_ccitt_16.crcValue, '04X')}\n")

### Configure State Machine - Coordination of the flashing process

In [3]:
# ********** Create State Machine **********

import numpy as np

# 1_ Super-class for the States of the State-Machine
class SuperState( object ):
    def stateActions( self, input ):
        pass
    def stateTransition( self, input ):
        raise NotImplementedError()
    
# 2_ States implementation
class Idle_01( SuperState ):
    def stateTransition( self, input ):
        if input is not None and len(input) > 0 and input[0] == 0x80:
            print("--> Device found! Starting communication to flash SW + sending SW version to MCU <--")
            # Send the answer command "C0" + SW version (MAJOR,MINOR)
            cmm = bytes( bytes({0xC0}) + # Command 0xC0
                        (int(binaryFile.SW_MAJOR)).to_bytes(1,'big') + # SW Major Version
                        (int(binaryFile.SW_MINOR)).to_bytes(1,'big') + # SW Minor Version
                        bytes({0xFF})) # Don't care
            serialInterface.writeSerialBuffer(cmm.hex()) 
            return CheckHeaderAndEraseMem_02()
        
class CheckHeaderAndEraseMem_02(SuperState):
    def stateActions( self, input ):
        if input is not None and len(input) > 0 and input[0] == 0x81:
            print("--> Erasing flash memory - Sector ", input[1] , " erased! <--")
    def stateTransition( self, input ):
        if input is not None and len(input) > 0 and input[0] == 0x8E:
            print("--> SW version already flashed in MCU. No action! <--")
            return Idle_01()
        elif input is not None and len(input) > 0 and input[0] == 0x81 and input[2] == 0x01:
            print("--> Erasing flash memory - All sectors erased succesfully! <--")
            fileSizePacketsHex =  binaryFile.fileSizePackets.to_bytes(3,'big')
            cmm = bytes(bytes({0xC1}) + fileSizePacketsHex) # Command 0xC1 + packetNum
            serialInterface.writeSerialBuffer(cmm.hex()) 
            return Prepare2Flash_03()

class Prepare2Flash_03( SuperState ):
    def stateActions( self, input ):
        if input is not None and len(input) > 0 and input[0] == 0x81:
            print("--> Waiting MCU readiness to start to send binary packets <--")
    def stateTransition( self, input ):
        if input is not None and len(input) > 0 and input[0] == 0x82:
            print("--> Sw authorized to be flashed <--")
            return SendFlashData_04()
        
class SendFlashData_04( SuperState ):
    def __init__(self):
        self.packetNum = 0 # Counter of transmitted packets    
    def stateActions( self, input ):        
        if self.packetNum < binaryFile.fileSizePackets:
            serialInterface.writeSerialBuffer(binaryFile.filePackets[self.packetNum].hex(), echo=0) # Transmit packets (no echo)
            self.packetNum += 1 #Increment packet counter
        if np.remainder(self.packetNum, 250) == 0: # User feedback
            print("--> Transmitted packets: ", self.packetNum, " <--")
    def stateTransition( self, input ):
        if input is not None and len(input) > 0 and input[0] == 0x84: # End of Flashing - MCU returns 0x84
            print("--> Transmission finished - Transmitted ", self.packetNum, " of ", binaryFile.fileSizePackets, " <--")
            # Check the CRC of the flashed data
            if input[2] == binaryFile.CRC_MSB and input[3] ==  binaryFile.CRC_LSB:
                print("--> CRC matched - Data integrity ensured!")
            else:
                print("--> CRC not matched - expected - Data integrity not ensured!")
            return FlashDone_99()
        elif input is not None and len(input) > 0 and input[0] == 0x8F: # Error - MCU returned error
            print("--> Error! <--")
            return Error_98()
        
class FlashDone_99( SuperState ):
    def stateTransition( self, input ):
        if input is not None and len(input) > 0 and input[0] == 0x85: # Jumping to application
            print("--> Bootloader finished. Jumping to application <--")
            return Idle_01()

class Error_98( SuperState ):
    def stateTransition( self, input ):
        print("--> Process error. Aborting! <--")
        return Idle_01()

# 3_ Context class - the state-machine "engine"
class Context:
    def __init__(self, initial_state):
        self._state = initial_state
    def execute(self,input):
        self._state.stateActions(input)
        _next_state = self._state.stateTransition(input)
        if _next_state is not None:
            self._state = _next_state

### Main Program execution

In [4]:
import keyboard # using module keyboard

# ----- Initialize an instance of the SerialInterface -----
serialInterface = SerialInterface()

# ----- Import binary file to be flashed in MCU -----
binaryFile = BinaryFile()
binaryFile.loadBinary("../STM32_SnakeGameNokia/Debug/STM32_SnakeGameNokia.bin")

# ----- Declare instance of Context (State-Machine), starting in the initial state -----
context = Context(Idle_01())

# ********** Main Program Flow **********
print("--> SW Loader @ V1.0. Waiting for device communication <--")
print("--> Matheus Sozza @ 2023 <--")
attemptCntr = 0

while 1:
    serialString = serialInterface.readSerialBuffer(echo = 1)
    context.execute(serialString)
    if serialString == None:
        attemptCntr +=1
    
    # Manual delay (faster than time library)
    for i in range(5000):
        pass

    if keyboard.is_pressed('q') or attemptCntr >= 600000000000: # if key 'a' is pressed OR timeout
        print('Stop serial pooling!')
        serialInterface.closeSerialComm()
        break # finishing the loop

--> Loading Binary file to be flashed <--
--> File Size:  74448  <--
--> Nr. of packets to be transmitted:  18612  <--
--> Packets example:  20020000  -  08041f05  -  0804186d
--> SW Version:  00000001  -  00000002
--> CRC-CCITT (16-bit) is: 0xEE38

--> SW Loader @ V1.0. Waiting for device communication <--
--> Matheus Sozza @ 2023 <--
     RX->  80ffffff
--> Device found! Starting communication to flash SW + sending SW version to MCU <--
     TX->  c00102ff
     RX->  810600ff
--> Erasing flash memory - Sector  6  erased! <--
     RX->  810700ff
--> Erasing flash memory - Sector  7  erased! <--
     RX->  810800ff
--> Erasing flash memory - Sector  8  erased! <--
     RX->  810900ff
--> Erasing flash memory - Sector  9  erased! <--
     RX->  810a01ff
--> Erasing flash memory - Sector  10  erased! <--
--> Erasing flash memory - All sectors erased succesfully! <--
     TX->  c10048b4
     RX->  820048b4
--> Sw authorized to be flashed <--
--> Transmitted packets:  250  <--
--> Transmit